In [20]:
import os.path #Will help us verify that the file exists
from os import path
import gc #Garbage collector, will help clean memory 

def FASTAReader(_filePath: str, _kMerslength: int): #New function 
    _resultDictionarylist = [] 
    _sequence = []
    _sequences = []
    _line = None #Null, the variable will hold no value
    if not path.exists(_filePath): #Will check if the file exists 
        print('file does not exists : {0}'.format(_filePath)) #Will print that the file does not exist if it's not a correct format
    else:
        _fastaFile = open(_filePath, 'r') #Open file
        _line = _fastaFile.readline() #Will read line of Fasta file, will return line by line
        assert _line.startswith('>'),'Not a fasta file' #Will verify (assert) that the file starts with >, or else will return that the file is not a fasta file
        _seq_id = _line[1:].strip('\n\r') #The sequence ID is the first line of the file and will strip it
        _line = _fastaFile.readline() #Will read line of Fasta file, will return line by line
        while _line: 
            if _line.startswith('>'):
                _sequences.append((_seq_id,''.join(_sequence))) #Will append the sequence to the sequence ID which is the one that starts with >
                _seq_id = _line[1:].rstrip('\n\r')
                _sequence = []
            else:
                _sequence.append(_line.strip()) 
            _line = _fastaFile.readline() 
        _sequences.append((_seq_id,''.join(_sequence))) #Joins the sequence ID to the sequence
        _seq_id = _line[1:].rstrip('\n\r') 
        for _SequenceId, _currentSequence in _sequences: #This is the part that will create the kmers
            for _index in range(0, len(_currentSequence) - _kMerslength + 1):
                _kmerLower = _currentSequence[_index:(_index+_kMerslength)].strip().lower()
                _kmerUpper = _currentSequence[_index:(_index+_kMerslength)].strip().upper()
                _kmerItem = {   'SequenceId' : _SequenceId.strip(),'LowerKeyValue' : _kmerLower,'UpperValue': _kmerUpper,'IndexPosition': _index     }
                _resultDictionarylist.append(_kmerItem) #Dictionary with all of the info on the Kmers 
    _sequence = None
    _sequences = None
    _line = None
    gc.collect() #clean memory
    return _resultDictionarylist 

In [26]:
import pandas as _pandas
import numpy as _numpy

def RunKmerMatcher(_targetFastaFilePath: str, _queryFastaFilePath: str, _kMerslength: int): #Will match kmers

    _targetDictionaryList = FASTAReader(_targetFastaFilePath, _kMerslength) #will run the first function for the target
    _queryDictionaryList = FASTAReader(_queryFastaFilePath, _kMerslength) #will run the first function for the query

    _targetDF = _pandas.DataFrame(_targetDictionaryList) #will create a data frame from the data of the target kmers
    _queryDF = _pandas.DataFrame(_queryDictionaryList) #will create a data frame from the data of the query kmers

    _targetDF.rename(columns={  'SequenceId': 'Target_SequenceId', #renames columns
                                'UpperValue': 'Target_UpperValue',\
                                'IndexPosition': 'Target_IndexPosition'}, inplace=True)
    _queryDF.rename(columns={   'SequenceId': 'Query_SequenceId',\
                                'UpperValue': 'Query_UpperValue',\
                                'IndexPosition': 'Query_IndexPosition'}, inplace=True)

    _targetDF.set_index('LowerKeyValue') #sets index for the kmer sequences  
    _queryDF.set_index('LowerKeyValue') 

    _kmerMatchesDataFrame = _pandas.merge(  _queryDF,_targetDF,how='inner',left_on=['LowerKeyValue'],right_on=['LowerKeyValue']).reset_index() #will merge both data frames depending on the values the have in common
    #will reset values and clean memory
    _targetDictionaryList = None 
    _queryDictionaryList = None
    _targetDF = None
    _queryDF = None
    gc.collect()

    _columnsNamesSingleLine = ''
    for _columnName in _kmerMatchesDataFrame.columns: 
        _columnsNamesSingleLine = _columnsNamesSingleLine + ('{0}\t'.format(_columnName)) #will format column name into the variable {0}
    print(_columnsNamesSingleLine.strip())
    for _index, _row in _kmerMatchesDataFrame.iterrows(): #will format different info on the kmers into different variables os that the info for that kmer is printed
        print('{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}'.format(_index\
                                                                ,_row['Query_SequenceId']\
                                                                ,_row['LowerKeyValue']\
                                                                ,_row['Query_UpperValue']\
                                                                ,_row['Query_IndexPosition']
                                                                ,_row['Target_SequenceId']\
                                                                ,_row['Target_UpperValue']\
                                                                ,_row['Target_IndexPosition']))

    return _kmerMatchesDataFrame

In [27]:
#verify functions
_queryFP = "droYak2_seq.fa"
_targetFP = "subset.fa"
_k = 11
_dataFrame_Final = RunKmerMatcher(_targetFP, _queryFP, _k)

index	Query_SequenceId	LowerKeyValue	Query_UpperValue	Query_IndexPosition	Target_SequenceId	Target_UpperValue	Target_IndexPosition
0	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	ccaggatttgg	CCAGGATTTGG	10	STRG.12.1	CCAGGATTTGG	66
1	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aggatttggat	AGGATTTGGAT	12	STRG.24.1	AGGATTTGGAT	1316
2	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	ggatttggatt	GGATTTGGATT	13	STRG.24.1	GGATTTGGATT	1317
3	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	gatttggattt	GATTTGGATTT	14	STRG.24.1	GATTTGGATTT	1318
4	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	gatttgttgtg	GATTTGTTGTG	20	STRG.11.1	GATTTGTTGTG	24
5	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=no

1010	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	agtatctatat	AGTATCTATAT	7170	STRG.15.1	AGTATCTATAT	1125
1011	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	atttaaaggag	ATTTAAAGGAG	7203	STRG.14.1	ATTTAAAGGAG	655
1012	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tttaaaggagt	TTTAAAGGAGT	7204	STRG.22.1	TTTAAAGGAGT	884
1013	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aattcaaaaaa	AATTCAAAAAA	7220	STRG.48.1	AATTCAAAAAA	629
1014	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aattcaaaaaa	AATTCAAAAAA	7220	STRG.48.2	AATTCAAAAAA	629
1015	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	caaaaaacttt	CAAAAAACTTT	7224	STRG.22.1	CAAAAAACTTT	102
1016	droYak2_ct_UserTrack_3545_0 range=chr2L:

1770	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	gtgtacagaaa	GTGTACAGAAA	9640	STRG.20.2	GTGTACAGAAA	7682
1771	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tgtacagaaaa	TGTACAGAAAA	9641	STRG.20.1	TGTACAGAAAA	7683
1772	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tgtacagaaaa	TGTACAGAAAA	9641	STRG.20.2	TGTACAGAAAA	7683
1773	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	acagaaaagca	ACAGAAAAGCA	9644	STRG.42.2	ACAGAAAAGCA	580
1774	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	agaaaagcacc	AGAAAAGCACC	9646	STRG.21.2	AGAAAAGCACC	837
1775	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	agaaaagcacc	AGAAAAGCACC	9646	STRG.21.1	AGAAAAGCACC	837
1776	droYak2_ct_UserTrack_3545_0 range=chr2

2385	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	cgagaactgaa	CGAGAACTGAA	14244	STRG.48.1	CGAGAACTGAA	2116
2386	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	cgagaactgaa	CGAGAACTGAA	14244	STRG.48.2	CGAGAACTGAA	2116
2387	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	atatttttttg	ATATTTTTTTG	14255	STRG.31.1	ATATTTTTTTG	2770
2388	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	atatttttttg	ATATTTTTTTG	14255	STRG.31.2	ATATTTTTTTG	2770
2389	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	atatttttttg	ATATTTTTTTG	14255	STRG.31.3	ATATTTTTTTG	1680
2390	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	atttttttgcg	ATTTTTTTGCG	14257	STRG.13.2	ATTTTTTTGCG	10
2391	droYak2_ct_UserTrack_3545_0 ran

3103	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aaaatcgctgg	AAAATCGCTGG	17338	STRG.26.1	AAAATCGCTGG	1110
3104	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aaaatcgctgg	AAAATCGCTGG	17338	STRG.26.2	AAAATCGCTGG	2132
3105	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aaatcgctggc	AAATCGCTGGC	17339	STRG.26.1	AAATCGCTGGC	1111
3106	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aaatcgctggc	AAATCGCTGGC	17339	STRG.26.2	AAATCGCTGGC	2133
3107	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aaatcgctggc	AAATCGCTGGC	26835	STRG.26.1	AAATCGCTGGC	1111
3108	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aaatcgctggc	AAATCGCTGGC	26835	STRG.26.2	AAATCGCTGGC	2133
3109	droYak2_ct_UserTrack_3545_0 r

...

12891	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tttaactaaaa	TTTAACTAAAA	71653	STRG.21.1	TTTAACTAAAA	952
12892	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	ttaactaaaat	TTAACTAAAAT	71654	STRG.21.2	TTAACTAAAAT	953
12893	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	ttaactaaaat	TTAACTAAAAT	71654	STRG.21.1	TTAACTAAAAT	953
12894	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	atgctgctgca	ATGCTGCTGCA	71663	STRG.37.1	ATGCTGCTGCA	791
12895	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	ttcgcaagggt	TTCGCAAGGGT	71697	STRG.1.1	TTCGCAAGGGT	2388
12896	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	cactctgcacc	CACTCTGCACC	71725	STRG.18.1	CACTCTGCACC	18026
12897	droYak2_ct_UserTrack_3545_

13529	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tcatggagctg	TCATGGAGCTG	77323	STRG.23.3	TCATGGAGCTG	1849
13530	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tcatggagctg	TCATGGAGCTG	77323	STRG.23.1	TCATGGAGCTG	1622
13531	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tcatggagctg	TCATGGAGCTG	77323	STRG.23.2	TCATGGAGCTG	1622
13532	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	catggagctgg	CATGGAGCTGG	77324	STRG.23.3	CATGGAGCTGG	1850
13533	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	catggagctgg	CATGGAGCTGG	77324	STRG.23.1	CATGGAGCTGG	1623
13534	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	catggagctgg	CATGGAGCTGG	77324	STRG.23.2	CATGGAGCTGG	1623
13535	droYak2_ct_UserTrack_3

14332	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	agtttgcagtt	AGTTTGCAGTT	84750	STRG.41.3	AGTTTGCAGTT	103
14333	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	gtttgcagttt	GTTTGCAGTTT	84751	STRG.41.2	GTTTGCAGTTT	104
14334	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	gtttgcagttt	GTTTGCAGTTT	84751	STRG.41.3	GTTTGCAGTTT	104
14335	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tttgcagttta	TTTGCAGTTTA	84752	STRG.41.2	TTTGCAGTTTA	105
14336	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tttgcagttta	TTTGCAGTTTA	84752	STRG.41.3	TTTGCAGTTTA	105
14337	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aaaacttggag	AAAACTTGGAG	84849	STRG.5.1	AAAACTTGGAG	3960
14338	droYak2_ct_UserTrack_3545_0 

14943	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tagttctagtg	TAGTTCTAGTG	90605	STRG.18.1	TAGTTCTAGTG	4089
14944	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	agttctagtgg	AGTTCTAGTGG	90606	STRG.18.2	AGTTCTAGTGG	4036
14945	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	agttctagtgg	AGTTCTAGTGG	90606	STRG.18.1	AGTTCTAGTGG	4090
14946	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	cccgcacattg	CCCGCACATTG	90728	STRG.47.2	CCCGCACATTG	21
14947	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	cccgcacattg	CCCGCACATTG	90728	STRG.47.1	CCCGCACATTG	21
14948	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	ggctccgcctc	GGCTCCGCCTC	90767	STRG.59.3	GGCTCCGCCTC	1814
14949	droYak2_ct_UserTrack_3545_

15501	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	gattctctgat	GATTCTCTGAT	95936	STRG.18.1	GATTCTCTGAT	9446
15502	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	tctctgatgaa	TCTCTGATGAA	95939	STRG.36.1	TCTCTGATGAA	2637
15503	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	gaattcatcag	GAATTCATCAG	95947	STRG.31.1	GAATTCATCAG	961
15504	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	gaattcatcag	GAATTCATCAG	95947	STRG.31.2	GAATTCATCAG	961
15505	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aacaacagtcg	AACAACAGTCG	95992	STRG.18.2	AACAACAGTCG	4577
15506	droYak2_ct_UserTrack_3545_0 range=chr2L:900001-1000000 5'pad=0 3'pad=0 strand=+ repeatMasking=none	aacaacagtcg	AACAACAGTCG	95992	STRG.18.1	AACAACAGTCG	4631
15507	droYak2_ct_UserTrack_354